In [ ]:
import pickle
import numpy as np 
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import sys
import os
import random

import tensorflow as tf
#tf.config.run_functions_eagerly(True)
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Concatenate, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.utils import plot_model
import pydot 
import graphviz

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

%config Completer.use_jedi = False

In [ ]:
#def seed_everything(seed: int = 42):
#    random.seed(seed)
#    np.random.seed(seed)
#    os.environ["PYTHONHASHSEED"] = str(seed)
#    tf.random.set_seed(seed)

In [ ]:
#seed_everything(123)

## Data

In [ ]:
with open('data_mat.pickle', 'rb') as data:
    data_mat = pickle.load(data)
with open('data_mat_seged.pickle', 'rb') as data:
    data_mat_seged = pickle.load(data)
with open('mask_mat.pickle', 'rb') as data:
    mask_mat = pickle.load(data)    
with open('labels_mat.pickle', 'rb') as data:
    labels_mat = pickle.load(data)    

In [ ]:
with open('pairs_mat.pickle', 'rb') as data:
    pairs_mat = pickle.load(data)  
with open('pairs_mat2.pickle', 'rb') as data:
    pairs_mat2 = pickle.load(data)  

In [ ]:
index1 = random.sample(range(0, 1328), 1062)
index2 = list(set(range(0, 1328)) - set(index1))

In [ ]:
pairs_mat2_3c = []
for i in range(0, len(pairs_mat2)):
    pairs_mat2_3c.append(cv2.cvtColor(pairs_mat2[i][0], cv2.COLOR_GRAY2RGB))

In [ ]:
pairs_mat2_3c = np.array(pairs_mat2_3c)

In [ ]:
pairs_mat2_2 = []

for i in range(0, len(pairs_mat2_3c)):
    pairs_mat2_2 += [[pairs_mat2_3c[i], np.array(pairs_mat2[i][1])/255]]

In [ ]:
x_train = []
for i in index1:
    x_train.append(pairs_mat[i])
x_train2 = []
for i in index1:
    x_train2.append(pairs_mat2_2[i])
y_train = []
for i in index1:
    y_train.append(labels_mat[i])

x_test = []
for i in index2:
    x_test.append(pairs_mat[i])
x_test2 = []
for i in index2:
    x_test2.append(pairs_mat2_2[i])
y_test = []
for i in index2:
    y_test.append(labels_mat[i])

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(np.array(x_data)/255,np.array(y_data), test_size=0.2, random_state=123)

In [ ]:
x_train = np.array(x_train)/255
x_train2 = np.array(x_train2)
y_train = np.array(y_train)

x_test = np.array(x_test)/255
x_test2 = np.array(x_test2)
y_test = np.array(y_test)

In [ ]:
print(x_train2[0][0].min())
print(x_train2[0][0].max())

In [ ]:
f, ax = plt.subplots(1,2)
ax[0].imshow(x_train[820][0])
ax[1].imshow(x_train[820][1])
plt.title(y_train[820])

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
tr_t_a = []
tr_b_a = []

for i in range(0, len(x_train)):
    tr_t_a.append(x_train[i][0][0:int(x_train[0][0].shape[0]/2)])
    tr_b_a.append(x_train[i][0][int(x_train[0][0].shape[0]/2):x_train[0][0].shape[0]])
    
tr_t_b = []
tr_b_b = []

for i in range(0, len(x_train)):
    tr_t_b.append(x_train[i][1][0:int(x_train[0][1].shape[0]/2)])
    tr_b_b.append(x_train[i][1][int(x_train[0][1].shape[0]/2):x_train[0][1].shape[0]])
    
tr_t_a = np.array(tr_t_a)
tr_b_a = np.array(tr_b_a)
tr_t_b = np.array(tr_t_b)
tr_b_b = np.array(tr_b_b)

data_list = [tr_t_a,tr_b_a, tr_t_b,tr_b_b]

In [ ]:
ts_t_a = []
ts_b_a = []

for i in range(0, len(x_test)):
    ts_t_a.append(x_test[i][0][0:int(x_test[0][0].shape[0]/2)])
    ts_b_a.append(x_test[i][0][int(x_test[0][0].shape[0]/2):x_test[0][0].shape[0]])
    
ts_t_b = []
ts_b_b = []

for i in range(0, len(x_test)):
    ts_t_b.append(x_test[i][1][0:int(x_test[0][1].shape[0]/2)])
    ts_b_b.append(x_test[i][1][int(x_test[0][1].shape[0]/2):x_test[0][1].shape[0]])
    
ts_t_a = np.array(ts_t_a)
ts_b_a = np.array(ts_b_a)
ts_t_b = np.array(ts_t_b)
ts_b_b = np.array(ts_b_b)

In [ ]:
tr_t_a2 = []
tr_b_a2 = []

for i in range(0, len(x_train2)):
    tr_t_a2.append(x_train2[i][0][0:int(x_train2[0][0].shape[0]/2)])
    tr_b_a2.append(x_train2[i][0][int(x_train2[0][0].shape[0]/2):x_train2[0][0].shape[0]])
    
tr_t_b2 = []
tr_b_b2 = []

for i in range(0, len(x_train2)):
    tr_t_b2.append(x_train2[i][1][0:int(x_train2[0][1].shape[0]/2)])
    tr_b_b2.append(x_train2[i][1][int(x_train2[0][1].shape[0]/2):x_train2[0][1].shape[0]])
    
tr_t_a2 = np.array(tr_t_a2)
tr_b_a2 = np.array(tr_b_a2)
tr_t_b2 = np.array(tr_t_b2)
tr_b_b2 = np.array(tr_b_b2)

data_list2 = [tr_t_a2,tr_b_a2, tr_t_b2,tr_b_b2]

In [ ]:
ts_t_a2 = []
ts_b_a2 = []

for i in range(0, len(x_test2)):
    ts_t_a2.append(x_test2[i][0][0:int(x_test2[0][0].shape[0]/2)])
    ts_b_a2.append(x_test2[i][0][int(x_test2[0][0].shape[0]/2):x_test2[0][0].shape[0]])
    
ts_t_b2 = []
ts_b_b2 = []

for i in range(0, len(x_test2)):
    ts_t_b2.append(x_test2[i][1][0:int(x_test2[0][1].shape[0]/2)])
    ts_b_b2.append(x_test2[i][1][int(x_test2[0][1].shape[0]/2):x_test2[0][1].shape[0]])
    
ts_t_a2 = np.array(ts_t_a2)
ts_b_a2 = np.array(ts_b_a2)
ts_t_b2 = np.array(ts_t_b2)
ts_b_b2 = np.array(ts_b_b2)

## Function

In [ ]:
def w_2(img):
        top, bottom = img

        h,w = top.get_shape()[1:3]
        g_t = tf.math.reduce_sum(top)/(h*w)
        g_b = tf.math.reduce_sum(bottom)/(h*w)
        w_t = g_t / tf.math.reduce_sum((g_t, g_b))
        w_b = g_b / tf.math.reduce_sum((g_t, g_b))
        return [w_t, w_b]          

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, 1e-7))

In [ ]:
def mul_sca(x):
    return tf.multiply(x[0],x[1])

In [ ]:
def plot_roc_curve(fper, tper, label):
    plt.plot(fper, tper, color='blue', label = label)
    plt.plot([0, 1], [0, 1], color='black', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc = 4)

In [ ]:
def visualize(pairs, labels, to_show=6, num_col=3, predictions=None, test=False):
    """Creates a plot of pairs and labels, and prediction if it's test dataset.

    Arguments:
        pairs: Numpy Array, of pairs to visualize, having shape
               (Number of pairs, 2, 28, 28).
        to_show: Int, number of examples to visualize (default is 6)
                `to_show` must be an integral multiple of `num_col`.
                 Otherwise it will be trimmed if it is greater than num_col,
                 and incremented if if it is less then num_col.
        num_col: Int, number of images in one row - (default is 3)
                 For test and train respectively, it should not exceed 3 and 7.
        predictions: Numpy Array of predictions with shape (to_show, 1) -
                     (default is None)
                     Must be passed when test=True.
        test: Boolean telling whether the dataset being visualized is
              train dataset or test dataset - (default False).

    Returns:
        None.
    """

    # Define num_row
    # If to_show % num_col != 0
    #    trim to_show,
    #       to trim to_show limit num_row to the point where
    #       to_show % num_col == 0
    #
    # If to_show//num_col == 0
    #    then it means num_col is greater then to_show
    #    increment to_show
    #       to increment to_show set num_row to 1
    num_row = to_show // num_col if to_show // num_col != 0 else 1

    # `to_show` must be an integral multiple of `num_col`
    #  we found num_row and we have num_col
    #  to increment or decrement to_show
    #  to make it integral multiple of `num_col`
    #  simply set it equal to num_row * num_col
    to_show = num_row * num_col

    # Plot the images
    fig, axes = plt.subplots(num_row, num_col, figsize=(5, 5))
    for i in range(to_show):

        # If the number of rows is 1, the axes array is one-dimensional
        if num_row == 1:
            ax = axes[i % num_col]
        else:
            ax = axes[i // num_col, i % num_col]

        ax.imshow(np.concatenate([pairs[i][0], pairs[i][1]], axis=1), cmap="gray")
        ax.set_axis_off()
        if test:
            ax.set_title("True: {} | Pred: {:.5f}".format(labels[i], predictions[i][0]))
        else:
            ax.set_title("Label: {}".format(labels[i]))
    if test:
        plt.tight_layout(rect=(0, 0, 1.9, 1.9), w_pad=0.0)
    else:
        plt.tight_layout(rect=(0, 0, 1.5, 1.5))
    plt.show()

## Model

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

### pretrained VGG19 

#### for original image

In [ ]:
model_vgg = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [ ]:
model_vgg.trainable = False

In [ ]:
inputs = Input((128, 128, 3))
x = model_vgg(inputs)
flatten = Flatten()
x = flatten(x)
x = Dense(128, activation='relu', name = 'vgg_output')(x)

In [ ]:
model = Model(inputs = inputs, outputs=x)

#### for segmentation image

In [ ]:
model_vgg_s = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
model_vgg_s.trainable = False

In [ ]:
inputs = Input((128, 128, 3))
x = model_vgg_s(inputs)
flatten = Flatten()
x = flatten(x)
x = Dense(128, activation='relu', name = 'vgg_s_output')(x)

In [ ]:
model_s = Model(inputs = inputs, outputs=x)

### siamese networks

#### MP-CNN

##### for original image

In [ ]:
input_t_a = Input(shape=(128,128,3), name='top_input_a')
vector_output_t_a = model(input_t_a)

input_b_a = Input(shape=(128,128,3), name='bottom_input_a')
vector_output_b_a = model(input_b_a)

w_a = Lambda(w_2, name = 'weight_a')([input_t_a, input_b_a])

weighted_vaector_t_a = Lambda(mul_sca, name = 'weighted_vaector_t_a')([vector_output_t_a, w_a[0]])
weighted_vaector_b_a = Lambda(mul_sca, name = 'weighted_vaector_b_a')([vector_output_b_a, w_a[1]])

output_a = Concatenate(name='concatenation_a')([weighted_vaector_t_a, weighted_vaector_b_a])  
#output_a = Concatenate(name='concatenation_a')([vector_output_t_a*w_a[0], vector_output_b_a*w_a[1]])  

model_a = Model([input_t_a, input_b_a], output_a)

In [ ]:
input_t_b = Input(shape=(128,128,3), name='top_input_b')
vector_output_t_b = model(input_t_b)

input_b_b = Input(shape=(128,128,3), name='bottom_input_b')
vector_output_b_b = model(input_b_b)


w_b = Lambda(w_2, name = 'weight_b')([input_t_b, input_b_b])

weighted_vaector_t_b = Lambda(mul_sca, name = 'weighted_vaector_t_b')([vector_output_t_b, w_b[0]])
weighted_vaector_b_b = Lambda(mul_sca, name = 'weighted_vaector_b_b')([vector_output_b_b, w_b[1]])

output_b = Concatenate(name='concatenation_b')([weighted_vaector_t_b, weighted_vaector_b_b])  
#output_b = Concatenate(name='concatenation_b')([vector_output_t_b*w_b[0], vector_output_b_b*w_b[1]])

model_b = Model([input_t_b, input_b_b], output_b)

In [ ]:
x_a = model_a.output
x_b = model_b.output

distance_layer = Lambda(lambda x: euclidean_distance([x[0], x[1]]))
distance = distance_layer([x_a, x_b])
#x = Lambda(euclidean_distance, name='distance')([x_a, x_b])
output = Dense(1, activation='sigmoid', name = 'output')(distance)

model_final = Model([input_t_a, input_b_a, input_t_b, input_b_b], output)

In [ ]:
random.seed(1234)
opt = Adam(learning_rate=0.01)
model_final.compile(loss='binary_crossentropy', optimizer=opt, metrics = ['accuracy'])

In [ ]:
history = model_final.fit(data_list, y_train, epochs=200, batch_size=32, 
                          validation_split=0.2, callbacks = [es, mc]) 

In [ ]:
model_final.save_weights('mp_cnn_dl')

##### for segmentation image

In [ ]:
input_t_a2 = Input(shape=(128,128,3), name='top_input_a2')
vector_output_t_a2 = model_s(input_t_a2)

input_b_a2 = Input(shape=(128,128,3), name='bottom_input_a2')
vector_output_b_a2 = model_s(input_b_a2)

w_a2 = Lambda(w_2, name = 'weight_a2')([input_t_a2, input_b_a2])

weighted_vaector_t_a2 = Lambda(mul_sca, name = 'weighted_vaector_t_a2')([vector_output_t_a2, w_a2[0]])
weighted_vaector_b_a2 = Lambda(mul_sca, name = 'weighted_vaector_b_a2')([vector_output_b_a2, w_a2[1]])

output_a2 = Concatenate(name='concatenation_a2')([weighted_vaector_t_a2, weighted_vaector_b_a2])  
#output_a = Concatenate(name='concatenation_a')([vector_output_t_a*w_a[0], vector_output_b_a*w_a[1]])  

model_a2 = Model([input_t_a2, input_b_a2], output_a2)

In [ ]:
input_t_b2 = Input(shape=(128,128,3), name='top_input_b2')
vector_output_t_b2 = model_s(input_t_b2)

input_b_b2 = Input(shape=(128,128,3), name='bottom_input_b2')
vector_output_b_b2 = model(input_b_b2)


w_b2 = Lambda(w_2, name = 'weight_b2')([input_t_b2, input_b_b2])

weighted_vaector_t_b2 = Lambda(mul_sca, name = 'weighted_vaector_t_b2')([vector_output_t_b2, w_b2[0]])
weighted_vaector_b_b2 = Lambda(mul_sca, name = 'weighted_vaector_b_b2')([vector_output_b_b2, w_b2[1]])

output_b2 = Concatenate(name='concatenation_b')([weighted_vaector_t_b2, weighted_vaector_b_b2])  
#output_b = Concatenate(name='concatenation_b')([vector_output_t_b*w_b[0], vector_output_b_b*w_b[1]])

model_b2 = Model([input_t_b2, input_b_b2], output_b2)

In [ ]:
x_a2 = model_a2.output
x_b2 = model_b2.output

distance_layer = Lambda(lambda x: euclidean_distance([x[0], x[1]]))
distance2 = distance_layer([x_a2, x_b2])
#x = Lambda(euclidean_distance, name='distance')([x_a, x_b])
output2 = Dense(1, activation='sigmoid', name = 'output')(distance2)

model_final_s = Model([input_t_a2, input_b_a2, input_t_b2, input_b_b2], output2)

In [ ]:
model_final_s.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics = ['accuracy'])

In [ ]:
history_s = model_final_s.fit(data_list, y_train, epochs=200, batch_size=100) 

In [ ]:
model_final_s.save_weights('mp_cnn_s_dl')

#### original siamese

##### for original image

In [ ]:
x_train_1 = np.array(x_train[:, 0])
x_train_2 = np.array(x_train[:, 1])

In [ ]:
model_vgg_o = VGG19(weights='imagenet', include_top=False, input_shape=(256, 128, 3))

In [ ]:
model_vgg_o.trainable = False

In [ ]:
inputs_o = Input((256, 128, 3))

x_o = model_vgg_o(inputs_o)
x_o = flatten(x_o)
x_o = Dense(256, activation='relu', name = 'output')(x_o)
embedding_network = Model(inputs_o, x_o)

input_o_1 = Input((256, 128, 3))
input_o_2 = Input((256, 128, 3))

tower_1 = embedding_network(input_o_1)
tower_2 = embedding_network(input_o_2)

merge_layer = Lambda(euclidean_distance)([tower_1, tower_2])
output_layer = Dense(1, activation="sigmoid")(merge_layer)
siamese = Model(inputs=[input_o_1, input_o_2], outputs=output_layer)

In [ ]:
random.seed(1234)

siamese.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics = ['accuracy'], )

In [ ]:
history_o = siamese.fit([x_train_1, x_train_2], y_train,epochs=200,batch_size=32, 
                          validation_split=0.2, callbacks = [es, mc]) 

In [ ]:
siamese.save_weights('siamese_dl')

##### for segmentation image

In [ ]:
model_vgg_o_s = VGG19(weights='imagenet', include_top=False, input_shape=(256, 128, 3))
model_vgg_o_s.trainable = False

In [ ]:
inputs_o2 = Input((256, 128, 3))

x_o2 = model_vgg_o_s(inputs_o2)
x_o2 = flatten(x_o2)
x_o2 = Dense(256, activation='relu', name = 'output')(x_o2)
embedding_network2 = Model(inputs_o2, x_o2)

input_o_1_2 = Input((256, 128, 3))
input_o_2_2 = Input((256, 128, 3))

tower_1_2 = embedding_network2(input_o_1_2)
tower_2_2 = embedding_network2(input_o_2_2)

merge_layer2 = Lambda(euclidean_distance)([tower_1_2, tower_2_2])
output_layer2 = Dense(1, activation="sigmoid")(merge_layer2)
siamese_s = Model(inputs=[input_o_1_2, input_o_2_2], outputs=output_layer2)

In [ ]:
x_train_1_2 = np.array(x_train2[:, 0])
x_train_2_2 = np.array(x_train2[:, 1])

In [ ]:
random.seed(1234)

siamese_s.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics = ['accuracy'],)

In [ ]:
history_o_s = siamese_s.fit([x_train_1_2, x_train_2_2], y_train, epochs=200,batch_size=32, 
                          validation_split=0.2, callbacks = [es, mc]) 

In [ ]:
siamese_s.save_weights('siamese_s_dl')

In [ ]:
siamese_s.save('siamese_s_dl_model')

### result

In [ ]:
results = model_final.evaluate([ts_t_a,ts_b_a, ts_t_b,ts_b_b], y_test)
y_pred_test = model_final.predict([ts_t_a,ts_b_a, ts_t_b,ts_b_b])
print("test loss, test acc:", results)

In [ ]:
results_s = model_final_s.evaluate([ts_t_a2,ts_b_a2, ts_t_b2,ts_b_b2], y_test)
y_pred_test_s = model_final_s.predict([ts_t_a2,ts_b_a2, ts_t_b2,ts_b_b2])
print("test loss, test acc:", results_s)

In [ ]:
x_test_1 = np.array(x_test[:, 0])
x_test_2 = np.array(x_test[:, 1])

results_o = siamese.evaluate([x_test_1, x_test_2], y_test)
y_pred_test_o = siamese.predict([x_test_1, x_test_2])
print("test loss, test acc:", results_o)

In [ ]:
x_test_1_2 = np.array(x_test2[:, 0])
x_test_2_2 = np.array(x_test2[:, 1])

results_o_s = siamese_s.evaluate([x_test_1_2, x_test_2_2], y_test)
y_pred_test_o_s = siamese_s.predict([x_test_1_2, x_test_2_2])
print("test loss, test acc:", results_o_s)

In [ ]:
test_auc = roc_auc_score(y_score=y_pred_test, y_true=y_test)
test_auc_s = roc_auc_score(y_score=y_pred_test_s, y_true=y_test)
test_auc_o = roc_auc_score(y_score=y_pred_test_o, y_true=y_test)
test_auc_o_s = roc_auc_score(y_score=y_pred_test_o_s, y_true=y_test)

print(round(test_auc,5))
print(round(test_auc_s,5))
print(round(test_auc_o,5))
print(round(test_auc_o_s,5))

In [ ]:
fper, tper, thresholds = roc_curve(y_test, y_pred_test)
fper_s, tper_s, thresholds = roc_curve(y_test, y_pred_test_s)
fper_o, tper_o, thresholds = roc_curve(y_test, y_pred_test_o)
fper_o_s, tper_o_s, thresholds = roc_curve(y_test, y_pred_test_o_s)

In [ ]:
plt.plot(fper, tper, label = round(test_auc,5))
plt.plot(fper_s, tper_s, label = round(test_auc_s,5))
plt.plot(fper_o, tper_o, label = round(test_auc_o,5))
plt.plot(fper_o_s, tper_o_s, label = round(test_auc_o_s,5))

plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc = 4)

In [ ]:
df_1 = pd.DataFrame(y_test, columns=['real'])
df_2 = pd.DataFrame(y_pred_test.ravel(), columns=['pred'])

In [ ]:
df = pd.concat([df_1, df_2], axis=1)

In [ ]:
df

In [ ]:
plt.figure(figsize = (8,6))
sns.kdeplot(data=df, x = 'pred', hue= 'real', fill = True)
#plt.xlim([0,1])
plt.legend(labels = ['M', 'NM'],loc = 'upper left')
#matching : 1, non-matching : 0

In [ ]:
plt.figure(figsize = (8,6))
sns.displot(data=df, x = 'pred', hue= 'real', fill = True)
plt.legend(labels = ['M', 'NM'],loc = 'upper right')
#matching : 1, non-matching : 0

In [ ]:
print(y_pred_test.ravel().min())
print(y_pred_test.ravel().max())

In [ ]:
df_1_o = pd.DataFrame(y_test, columns=['real'])
df_2_o = pd.DataFrame(y_pred_test_o.ravel(), columns=['pred'])
df_o = pd.concat([df_1_o, df_2_o], axis=1)

In [ ]:
df_o

In [ ]:
plt.figure(figsize = (8,6))
sns.kdeplot(data=df_o, x = 'pred', hue= 'real', fill = True)
plt.legend(labels = ['M', 'NM'],loc = 'upper right')
#matching : 1, non-matching : 0

In [ ]:
print(y_pred_test_o.ravel().min())
print(y_pred_test_o.ravel().max())

In [ ]:
index = [i for i, x in enumerate(y_test!=(y_pred_test.ravel()>0.5)) if x]
len(index)

In [ ]:
[i for i, x in enumerate(y_test!=(y_pred_test.ravel()>0.5)) if x]

In [ ]:
index_o = [i for i, x in enumerate(y_test!=(y_pred_test_o.ravel()>0.5)) if x]
len(index_o)

In [ ]:
predictions = model_final.predict([ts_t_a,ts_b_a, ts_t_b,ts_b_b])

In [ ]:
visualize(x_test, y_test, to_show=4, predictions=predictions, test=True, num_col=4)

In [ ]:
predictions_o =siamese.predict([x_test_1, x_test_2])

In [ ]:
visualize(x_test, y_test, to_show=4, predictions=predictions_o, test=True, num_col=4)

In [ ]:
visualize(x_test[index], y_test[index], to_show=6, predictions=predictions[index], test=True, num_col=3)

In [ ]:
visualize(x_test[index_o], y_test[index_o], to_show=6, predictions=predictions_o[index_o], test=True)